In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Exercises Ensemble models

1.- Use dataset Autoprice to train several regression models. You can choose your favourites among **Bagging, Stacking and Boosting** models. Train at least 2 regressions models with the best hyperparameters (You can find the best ones using GridSearch)

In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/drive/MyDrive/Advanced Data Analytics UAX-20231025T214223Z-001/Advanced Data Analytics UAX/data/2.-Complex ML/autoprice.csv', sep=';',encoding='utf-8')

X=df.drop(columns=['price'])
y=df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

original_index_train = X_train.index
original_index_test = X_test.index

scaler = StandardScaler()
car_scaler = scaler.fit(X_train)
train_car_scaler = car_scaler.transform(X_train)
test_car_scaler = car_scaler.transform(X_test)

# Crear un nuevo DataFrame con las características estandarizadas
X_train_scaler = pd.DataFrame(train_car_scaler, columns=X_train.columns, index = original_index_train)
X_test_scaler = pd.DataFrame(test_car_scaler, columns=X_test.columns, index = original_index_test)



In [54]:
from sklearn.ensemble import BaggingRegressor, StackingRegressor, GradientBoostingRegressor,AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

bagging_params = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0],
}

boosting_params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 4, 5]
}

boosting_params2 = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
}

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid=boosting_params, n_jobs=-1, cv=cv, scoring = 'neg_mean_absolute_error')
# execute the grid search
grid_result = grid_search.fit(X_train_scaler, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# define the grid search procedure
grid_search2 = GridSearchCV(estimator=AdaBoostRegressor(), param_grid=boosting_params2, n_jobs=-1, cv=cv,scoring = 'neg_mean_absolute_error')
# execute the grid search
grid_result2 = grid_search2.fit(X_train_scaler, y_train)

# summarize the best score and configuration
print("\nBest: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))

# summarize all scores that were evaluated
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


# define the grid search procedure
grid_search3 = GridSearchCV(estimator=BaggingRegressor(), param_grid=bagging_params, n_jobs=-1, cv=cv,scoring = 'neg_mean_absolute_error')
# execute the grid search
grid_result3 = grid_search3.fit(X_train_scaler, y_train)

# summarize the best score and configuration
print("\nBest: %f using %s" % (grid_result3.best_score_, grid_result3.best_params_))

# summarize all scores that were evaluated
means = grid_result3.cv_results_['mean_test_score']
stds = grid_result3.cv_results_['std_test_score']
params = grid_result3.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))




Best: -0.157233 using {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 50}
-2.885118 (0.282030) with: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
-1.960546 (0.192711) with: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
-1.403471 (0.151189) with: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150}
-2.823729 (0.280798) with: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50}
-1.872743 (0.177302) with: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}
-1.286294 (0.133606) with: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 150}
-2.784784 (0.273049) with: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
-1.802034 (0.183399) with: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
-1.207471 (0.118870) with: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 150}
-0.591524 (0.093958) with: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
-0.453938 (0.089477) with: {'learning_rate':

In [55]:
from sklearn.metrics import mean_squared_error

best_bagging_params = {
    'n_estimators': 50,
    'max_samples': 1,
    'max_features': 1.0
}

best_boosting_params = {
    'n_estimators': 50,
    'learning_rate': 0.5,
    'max_depth': 5
}

# Entrenar los modelos con los mejores hiperparámetros
bagging_model = BaggingRegressor(**best_bagging_params)
boosting_model = GradientBoostingRegressor(**best_boosting_params)

bagging_model.fit(X_train_scaler, y_train)
boosting_model.fit(X_train_scaler, y_train)

# Evaluar el rendimiento en el conjunto de prueba
bagging_pred = bagging_model.predict(X_test_scaler)
boosting_pred = boosting_model.predict(X_test_scaler)

print("MAE: ", metrics.mean_absolute_error(y_test, bagging_pred).round(4))
print("MSE: ", metrics.mean_squared_error(y_test, bagging_pred).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, bagging_pred)).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(y_test, bagging_pred).round(4))
print("R2: ", metrics.r2_score(y_test, bagging_pred).round(4))

print("\nMAE: ", metrics.mean_absolute_error(y_test, boosting_pred).round(4))
print("MSE: ", metrics.mean_squared_error(y_test, boosting_pred).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, boosting_pred)).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(y_test, boosting_pred).round(4))
print("R2: ", metrics.r2_score(y_test, boosting_pred).round(4))

MAE:  4.2688
MSE:  30.0387
RMSE:  5.4808
MAPE:  0.4195
R2:  -0.0009

MAE:  0.2134
MSE:  0.6999
RMSE:  0.8366
MAPE:  0.0161
R2:  0.9767


2.- Use Breast-cancer dataset to train several Classification models. You can choose your favourites among Bagging, Stacking and Boosting models. Train at least 2 classification models with the best hyperparameters (You can find the best ones using GridSearch)

In [56]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR, SVC # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization

import warnings


In [57]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('/content/drive/MyDrive/Advanced Data Analytics UAX-20231025T214223Z-001/Advanced Data Analytics UAX/data/2.-Complex ML/breast-cancer.csv', sep=';',encoding='utf-8')

X=df.drop(columns=['diagnosis'])
y=df['diagnosis']
print(y.value_counts())
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
print(encoded_Y)

X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.2, random_state=42)

original_index_train = X_train.index
original_index_test = X_test.index

scaler = StandardScaler()
car_scaler = scaler.fit(X_train)
train_car_scaler = car_scaler.transform(X_train)
test_car_scaler = car_scaler.transform(X_test)

# Crear un nuevo DataFrame con las características estandarizadas
X_train_scaler = pd.DataFrame(train_car_scaler, columns=X_train.columns, index = original_index_train)
X_test_scaler = pd.DataFrame(test_car_scaler, columns=X_test.columns, index = original_index_test)




B    357
M    212
Name: diagnosis, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0

In [58]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

randomforest_params={
    'n_estimators' : [50, 100, 150],
    'criterion' : ['gini','entropy']
    }

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=boosting_params, n_jobs=-1, cv=cv, scoring = 'accuracy')
# execute the grid search
grid_result = grid_search.fit(X_train_scaler, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# define the grid search procedure
grid_search2 = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=boosting_params2, n_jobs=-1, cv=cv,scoring = 'accuracy')
# execute the grid search
grid_result2 = grid_search2.fit(X_train_scaler, y_train)

# summarize the best score and configuration
print("\nBest: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))

# summarize all scores that were evaluated
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


# define the grid search procedure
grid_search3 = GridSearchCV(estimator=RandomForestClassifier(), param_grid=randomforest_params, n_jobs=-1, cv=cv,scoring = 'accuracy')
# execute the grid search
grid_result3 = grid_search3.fit(X_train_scaler, y_train)

# summarize the best score and configuration
print("\nBest: %f using %s" % (grid_result3.best_score_, grid_result3.best_params_))

# summarize all scores that were evaluated
means = grid_result3.cv_results_['mean_test_score']
stds = grid_result3.cv_results_['std_test_score']
params = grid_result3.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.961905 using {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 100}
0.932601 (0.031047) with: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
0.936264 (0.033138) with: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
0.941392 (0.032017) with: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150}
0.934066 (0.031340) with: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50}
0.939927 (0.031305) with: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}
0.939194 (0.030524) with: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 150}
0.930403 (0.027116) with: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
0.939194 (0.026272) with: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
0.934066 (0.028088) with: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 150}
0.948718 (0.023626) with: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
0.953114 (0.022864) with: {'learning_rate': 0.1, 'max_

In [59]:
from tabulate import tabulate

best_boosting_params2 = {
    'n_estimators': 100,
    'learning_rate': 0.5
}

best_boosting_params = {
  'n_estimators': 50,
  'learning_rate': 0.5,
  'max_depth': 5
}

# Entrenar los modelos con los mejores hiperparámetros
boosting_model2 = AdaBoostClassifier(**best_boosting_params2)
boosting_model = GradientBoostingClassifier(**best_boosting_params)

boosting_model2.fit(X_train_scaler, y_train)
boosting_model.fit(X_train_scaler, y_train)

# Evaluar el rendimiento en el conjunto de prueba
boosting_pred2 = boosting_model2.predict(X_test_scaler)
boosting_pred = boosting_model.predict(X_test_scaler)

print("Train set score (Accuracy) =", boosting_model.score(X_train_scaler, y_train).round(4))
print("Test set score (Accuracy) =", boosting_model.score(X_test_scaler, y_test).round(4))

conf_mat = metrics.confusion_matrix(y_test, boosting_model.predict(X_test_scaler))
print(tabulate(conf_mat,headers = ['pred M','pred B'], showindex = ['real M','real B'],
               tablefmt = 'fancy_grid'))

print(metrics.classification_report(y_test, boosting_model.predict(X_test_scaler)))

print("\nTrain set score (Accuracy) =", boosting_model2.score(X_train_scaler, y_train).round(4))
print("Test set score (Accuracy) =", boosting_model2.score(X_test_scaler, y_test).round(4))

conf_mat = metrics.confusion_matrix(y_test, boosting_model2.predict(X_test_scaler))
print(tabulate(conf_mat,headers = ['pred M','pred B'], showindex = ['real M','real B'],
               tablefmt = 'fancy_grid'))

print(metrics.classification_report(y_test, boosting_model2.predict(X_test_scaler)))

Train set score (Accuracy) = 1.0
Test set score (Accuracy) = 0.9561
╒════════╤══════════╤══════════╕
│        │   pred M │   pred B │
╞════════╪══════════╪══════════╡
│ real M │       69 │        2 │
├────────┼──────────┼──────────┤
│ real B │        3 │       40 │
╘════════╧══════════╧══════════╛
              precision    recall  f1-score   support

           0       0.96      0.97      0.97        71
           1       0.95      0.93      0.94        43

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114


Train set score (Accuracy) = 1.0
Test set score (Accuracy) = 0.9649
╒════════╤══════════╤══════════╕
│        │   pred M │   pred B │
╞════════╪══════════╪══════════╡
│ real M │       69 │        2 │
├────────┼──────────┼──────────┤
│ real B │        2 │       41 │
╘════════╧══════════╧══════════╛
              precision    recall  f1-score   support

           0       

## Exercises SVM_Models

3.- Using the code and the datasets of the theory notebooks (SVM_Models.ipynb) and **Grid Search**, find the best hyperparameters for the SVM and for the SVR models

In [60]:
warnings.filterwarnings("ignore")
df = pd.read_csv('/content/drive/MyDrive/Advanced Data Analytics UAX-20231025T214223Z-001/Advanced Data Analytics UAX/data/2.-Complex ML/Real estate.csv', encoding='utf-8')

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df[['X3 distance to the nearest MRT station']],
                                                    df['Y house price of unit area'], train_size = 0.8, random_state = 0)
X_train.head(3)

,X3 distance to the nearest MRT station
302,2288.0110
20,2275.8770
303,439.7105


In [62]:
model1 = LinearRegression()
lr = model1.fit(X_train, y_train)

SVR_model = SVR()

# define the grid of values to search

svr_params={
    'kernel' :['rbf'],
    'C' : [0.1, 1.0, 10],
    'epsilon':[0.001, 0.01, 0.1, 10],
    'gamma':[0.001, 0.01, 1, 10]
    }


# define the evaluation procedure
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search = GridSearchCV(estimator=SVR_model, param_grid=svr_params, n_jobs=-1, cv=cv, scoring='neg_mean_absolute_percentage_error')
# execute the grid search

grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -0.185869 using {'C': 10, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
-0.354668 (0.050076) with: {'C': 0.1, 'epsilon': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
-0.360028 (0.050554) with: {'C': 0.1, 'epsilon': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
-0.364686 (0.050812) with: {'C': 0.1, 'epsilon': 0.001, 'gamma': 1, 'kernel': 'rbf'}
-0.364997 (0.050702) with: {'C': 0.1, 'epsilon': 0.001, 'gamma': 10, 'kernel': 'rbf'}
-0.354663 (0.050065) with: {'C': 0.1, 'epsilon': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
-0.360030 (0.050557) with: {'C': 0.1, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
-0.364706 (0.050820) with: {'C': 0.1, 'epsilon': 0.01, 'gamma': 1, 'kernel': 'rbf'}
-0.365023 (0.050719) with: {'C': 0.1, 'epsilon': 0.01, 'gamma': 10, 'kernel': 'rbf'}
-0.354629 (0.049998) with: {'C': 0.1, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
-0.360151 (0.050480) with: {'C': 0.1, 'epsilon': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
-0.364950 (0.050932) with: {'C': 0.1, 'epsilon': 0.1, 

In [63]:
model2 = SVR(kernel='rbf', C=10, epsilon=0.1, gamma = 0.001) # set kernel and hyperparameters
svr = model2.fit(X_train, y_train)

print("MAE: ", metrics.mean_absolute_error(y_test, model2.predict(X_test)).round(4))
print("MSE: ", metrics.mean_squared_error(y_test, model2.predict(X_test)).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, model2.predict(X_test))).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(y_test, model2.predict(X_test)).round(4))
print("R2: ", metrics.r2_score(y_test, model2.predict(X_test)).round(4))

def plot(df, model1, model2):
  # Create a scatter plot
  fig = px.scatter(df, x=df['X3 distance to the nearest MRT station'], y=df['Y house price of unit area'],
                  opacity=0.8, color_discrete_sequence=['black'])
  X = df['X3 distance to the nearest MRT station']
  # Predict y values for our set of X values
  x_range = np.linspace(X.min(), X.max(), 100)
  y_lr = model1.predict(x_range.reshape(-1, 1)) # Linear regression
  y_svr = model2.predict(x_range.reshape(-1, 1)) # SVR

  # Add a best-fit line
  fig.add_traces(go.Scatter(x=x_range, y=y_lr, name='Linear Regression', line=dict(color='limegreen')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr, name='Support Vector Regression', line=dict(color='red')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr+10, name='+epsilon', line=dict(color='red', dash='dot')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr-10, name='-epsilon', line=dict(color='red', dash='dot')))

  # Change chart background color
  fig.update_layout(dict(plot_bgcolor = 'white'))

  # Update axes lines
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                  zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                  showline=True, linewidth=1, linecolor='black')

  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                  zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                  showline=True, linewidth=1, linecolor='black')

  # Set figure title
  fig.update_layout(title=dict(text="House Price Based on Distance from the Nearest MRT",
                              font=dict(color='black')))
  # Update marker size
  fig.update_traces(marker=dict(size=3))

  fig.show()

plot(df, model1, model2)

MAE:  5.7194
MSE:  63.1861
RMSE:  7.949
MAPE:  0.1729
R2:  0.6362


In [64]:
df0 = pd.read_csv('/content/drive/MyDrive/Advanced Data Analytics UAX-20231025T214223Z-001/Advanced Data Analytics UAX/data/2.-Complex ML/games.csv', sep = ';')
df0['rating_difference']=df0['white_rating']-df0['black_rating']
df0['white_win']=df0['winner'].apply(lambda x: 1 if x=='white' else 0)
df = df0[['rating_difference', 'turns','white_win']]

X=df[['rating_difference', 'turns']]#
y=df['white_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.head(3)

,rating_difference,turns
17843,-678,86
5518,-42,78
7699,61,100


In [65]:
SVC_model = SVC()

# define the grid of values to search

svc_params={
    'C' : [0.1, 1.0],
    'gamma':[0.001, 0.01, 1]
    }


# define the evaluation procedure
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search_svc = GridSearchCV(estimator=SVC_model, param_grid=svc_params, n_jobs=-1, cv=cv, scoring='accuracy')
# execute the grid search

grid_result = grid_search_svc.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.641780 using {'C': 0.1, 'gamma': 0.001}
0.641780 (0.007013) with: {'C': 0.1, 'gamma': 0.001}
0.630251 (0.005855) with: {'C': 0.1, 'gamma': 0.01}
0.494516 (0.003804) with: {'C': 0.1, 'gamma': 1}
0.639578 (0.007548) with: {'C': 1.0, 'gamma': 0.001}
0.627840 (0.006749) with: {'C': 1.0, 'gamma': 0.01}
0.579957 (0.008930) with: {'C': 1.0, 'gamma': 1}


In [66]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

model = SVC(kernel='rbf', probability=True, C=0.1, gamma=0.0001)
SVC_model = model.fit(X_train, y_train)

pred_labels_tr = SVC_model.predict(X_train)
pred_labels_ts = SVC_model.predict(X_test)

print("Train set score (Accuracy) =", SVC_model.score(X_train, y_train).round(4))
print("Test set score (Accuracy) =", SVC_model.score(X_test, y_test).round(4))

conf_mat = confusion_matrix(y_test, SVC_model.predict(X_test))
print(tabulate(conf_mat,headers = ['pred win No','pred win Yes'], showindex = ['real win No','real win Yes'],
               tablefmt = 'fancy_grid'))

print(classification_report(y_test, SVC_model.predict(X_test)))

def Plot_3D(X, X_test, y_test, clf):

    # Specify a size of the mesh to be used
    mesh_size = 5
    margin = 1

    # # Create a mesh grid on which we will run our model
    x_min, x_max = X.iloc[:, 0].fillna(X.mean()).min() - margin, X.iloc[:, 0].fillna(X.mean()).max() + margin
    y_min, y_max = X.iloc[:, 1].fillna(X.mean()).min() - margin, X.iloc[:, 1].fillna(X.mean()).max() + margin
    xrange = np.arange(x_min, x_max, mesh_size)
    yrange = np.arange(y_min, y_max, mesh_size)
    xx, yy = np.meshgrid(xrange, yrange)

    # Calculate predictions on grid
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)

    # Create a 3D scatter plot with predictions
    fig = px.scatter_3d(x=X_test['rating_difference'], y=X_test['turns'], z=y_test,
                     opacity=0.8, color_discrete_sequence=['black'])

    # Set figure title and colors
    fig.update_layout(#title_text="Scatter 3D Plot with SVM Prediction Surface",
                      paper_bgcolor = 'white',
                      scene = dict(xaxis=dict(backgroundcolor='white',
                                              color='black',
                                              gridcolor='#f0f0f0'),
                                   yaxis=dict(backgroundcolor='white',
                                              color='black',
                                              gridcolor='#f0f0f0'
                                              ),
                                   zaxis=dict(backgroundcolor='lightgrey',
                                              color='black',
                                              gridcolor='#f0f0f0',
                                              )))
    # Update marker size
    fig.update_traces(marker=dict(size=1))

    # Add prediction plane
    fig.add_traces(go.Surface(x=xrange, y=yrange, z=Z, name='SVM Prediction',
                              colorscale='RdBu', showscale=False,
                              contours = {"z": {"show": True, "start": 0.2, "end": 0.8, "size": 0.05}}))
    fig.show()

Plot_3D(X, X_test, y_test, SVC_model)

Train set score (Accuracy) = 0.6475
Test set score (Accuracy) = 0.6555
╒══════════════╤═══════════════╤════════════════╕
│              │   pred win No │   pred win Yes │
╞══════════════╪═══════════════╪════════════════╡
│ real win No  │          1431 │            593 │
├──────────────┼───────────────┼────────────────┤
│ real win Yes │           789 │           1199 │
╘══════════════╧═══════════════╧════════════════╛
              precision    recall  f1-score   support

           0       0.64      0.71      0.67      2024
           1       0.67      0.60      0.63      1988

    accuracy                           0.66      4012
   macro avg       0.66      0.66      0.65      4012
weighted avg       0.66      0.66      0.65      4012

